In [3]:
import plotly
import cufflinks
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
import cvxopt as opt
from cvxopt import blas, solvers

py.init_notebook_mode(connected=True)

plotly.__version__

'5.15.0'

In [4]:
#from pandas_datareader import data as wb
#ibov = wb.DataReader('^BVSP', data_source = 'yahoo', start = '2014-01-01', end = '2021-05-03')['Adj Close']
#type(ibov)


In [5]:
np.random.seed(123)

# Turn off progress printing 
solvers.options['show_progress'] = False

## NUMBER OF ASSETS
n_assets = 4

## NUMBER OF OBSERVATIONS
n_obs = 1000

return_vec = np.random.randn(n_assets, n_obs)

In [6]:
data = [go.Scatter(y=asset.T) for asset in return_vec];
fig = go.Figure(data=data)
fig.show()

In [7]:

def rand_weights(n):
    k = np.random.rand(n)
    return k / sum(k)

print(rand_weights(n_assets))
print(rand_weights(n_assets))
print(rand_weights(n_assets))

[0.54066805 0.2360283  0.11660484 0.1066988 ]
[0.27638339 0.03006307 0.47850085 0.21505269]
[0.1418152  0.32120599 0.15426636 0.38271245]


In [8]:
def random_portfolio(returns):

    risk_free_rate = 0.02
    
    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    
    mu = (w * p.T) * 252

    sigma = np.sqrt(w * C * w.T) * np.sqrt(252)
    sharpe = (mu - risk_free_rate) / sigma 
    
    return mu, sigma, sharpe

In [9]:
n_portfolios = 500
means, stds, sharpe = np.column_stack([
    random_portfolio(return_vec) 
    for _ in range(n_portfolios)
])

In [10]:
l = len(stds.flatten())
data = [go.Scatter(
            x=stds.flatten(),
            y=means.flatten(),
            mode='markers',            
             marker=dict(
                    color=sharpe.flatten(),
                    showscale=True,
                    cmax=sharpe.flatten().max(),
                    cmin=0,                 
                    )       
        )
       ]

title_text = "Média e desvio padrão dos retornos para os portfólios gerados"
title={'text': title_text, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}

fig = go.Figure(data=data)

fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            yaxis_title="Média",
            xaxis_title="Desvio padrão",                  
            plot_bgcolor='rgba(0,0,0,0)'              
                 )
                  

fig.show()

In [11]:
def optimal_portfolio(returns):
    n = len(returns)
    returns = np.asmatrix(returns)
    
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
                  for mu in mus]
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) * 252 for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) * np.sqrt(252) for x in portfolios]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    
    return np.asarray(wt), returns, risks

weights, returns, risks = optimal_portfolio(return_vec)

In [12]:
data = [go.Scatter(
            x=stds.flatten(),
            y=means.flatten(),
            mode='markers',            
             marker=dict(
                    color=sharpe.flatten(),
                    showscale=True,
                    cmax=sharpe.flatten().max(),
                    cmin=0,                 
                    )       
        ),
        go.Scatter(
            x=risks,
            y=returns,
            mode='lines+markers',            
        ),    
        go.Scatter(
            x=stds[sharpe.argmax()],
            y=means[sharpe.argmax()],
            mode='markers',            
        ),                    
       ]

title_text = "Risco x Retorno para os portfólios gerados"
title={'text': title_text, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}

fig = go.Figure(data=data)

fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            yaxis_title="Retorno",
            xaxis_title="Risco",                  
            plot_bgcolor='rgba(0,0,0,0)'              
                 )
                  

fig.show()